In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

# ファイル名を実際の訓練・テストデータファイル名に置き換えてください
train_data_xgb = pd.read_csv('selected_train_set.csv')
test_data_xgb = pd.read_csv('selected_test_set.csv')
index_col=['id']
# 特徴量（X）と目標変数（y）の設定
X_train_xgb = train_data_xgb.drop(columns=['PassengerId','Survived'])  # 'target_column'を目標変数の列名に置き換えてください
y_train_xgb = train_data_xgb['Survived']
X_test_xgb = test_data_xgb.drop(columns=['PassengerId','Survived'])
y_test_xgb = test_data_xgb['Survived']



from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import xgboost as xgb

# XGBoostモデルのインスタンス作成
xgb_model = xgb.XGBClassifier(eval_metric='logloss')

# パラメータの分布を設定
param_dist = {
    'n_estimators': [100, 200, 300],       # 試すブーストラウンド数
    'learning_rate': [0.01, 0.05, 0.1],    # 試す学習率
    'max_depth': [3, 4, 5],                # 木の最大深さ
    'subsample': [0.8, 1.0],               # サブサンプリング比率
    'colsample_bytree': [0.6, 0.8, 1.0],   # 特徴量のサブサンプリング比率
    'gamma': [0, 0.1, 0.2]                 # 正則化項
}

# RandomizedSearchCVの設定
random_search = RandomizedSearchCV(
    estimator=xgb_model, param_distributions=param_dist, n_iter=10,  # 試行回数
    scoring='accuracy', cv=5, random_state=42, n_jobs=-1
)

# 最適なパラメータの探索
random_search.fit(X_train_xgb, y_train_xgb)

# 最適なパラメータの表示
print(f"Best parameters found: {random_search.best_params_}")
print(f"Best cross-validation accuracy: {random_search.best_score_:.2f}")

# 最適なパラメータでのモデル評価
best_xgb_model = random_search.best_estimator_


In [ ]:
# クラスの不均衡を考慮した重みの計算
ratio = y_train_xgb.value_counts()[0] / y_train_xgb.value_counts()[1]  # 負例/正例の比率


# DMatrixに変換
dtrain_xgb = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
for i in range(10):
# パラメータ設定
    params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'learning_rate': 0.1,
    'max_depth': 5,
    'subsample': 1.0,
    'colsample_bytree': 1.0,
    'gamma': 0.1
}

# クロスバリデーション
cv_results = xgb.cv(
    params=params,
    dtrain=dtrain_xgb,
    num_boost_round=200,  # 最大のブーストラウンド数
    nfold=5,  # 5分割でのクロスバリデーション
    early_stopping_rounds=10,  # 10回のラウンドで改善がなければ停止
    metrics='logloss',  # 評価指標
    seed=42,
    verbose_eval=False
)

# 最適なブーストラウンド数を取得
optimal_boost_rounds = len(cv_results)
print(f"Optimal number of boosting rounds: {optimal_boost_rounds}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

# クロスバリデーションで得た最適なブーストラウンド数
optimal_boost_rounds = 28  # ここで得た値

# クラスの不均衡を考慮した重みの計算
ratio = y_train_xgb.value_counts()[0] / y_train_xgb.value_counts()[1]  # 負例/正例の比率

# 10回ループの準備
accuracies_xgb = []  # 精度を格納するリスト


for i in range(10):
    # 訓練-検証の分割
    X_train_split_xgb, X_val_split_xgb, y_train_split_xgb, y_val_split_xgb = train_test_split(X_train_xgb, y_train_xgb, test_size=0.2, random_state=42)

#以下に新しく提案されたコードを各
    # DMatrixに変換
    dtrain_xgb = xgb.DMatrix(X_train_split_xgb, label=y_train_split_xgb)
    dval_xgb = xgb.DMatrix(X_val_split_xgb, label=y_val_split_xgb)
    dtest_xgb = xgb.DMatrix(X_test_xgb)

    params = {
        'objective':'binary:logistic',
        'eval_metric':'logloss',
        'random_state':42,
        'learning_rate':0.1,
        'max_depth':5,
        'subsample':1.0,
        'colsample_bytree':1.0,
        'gamma':0.1
}

    
    # アーリーストッピングでモデルを訓練
    evals_xgb = [(dtrain_xgb, 'train'), (dval_xgb, 'eval')]
    model_xgb = xgb.train(params, dtrain_xgb, num_boost_round=optimal_boost_rounds, early_stopping_rounds=10, evals=evals_xgb, verbose_eval=False)
    #num_boost_roundを100(0.51)から200に、learn_rateを0.1から0.05に（過学習抑制）、max_depthを3からまずは4に変更
    # 予測
    y_test_pred_xgb = model_xgb.predict(dtest_xgb)
    y_test_pred_xgb = [1 if pred > 0.5 else 0 for pred in y_test_pred_xgb]  # 二値分類として閾値0.5で予測値を変換
    accuracy_xgb = accuracy_score(y_test_xgb, y_test_pred_xgb)
    accuracies_xgb.append(accuracy_xgb)

# 平均精度を計算
average_accuracy_xgb = np.mean(accuracies_xgb)
print(f'Average Accuracy over 10 runs: {average_accuracy_xgb:.2f}')

In [ ]:
import matplotlib.pyplot as plt
# 特徴量の重要度を取得 (weightとgain)
importance_weight_xgb = model_xgb.get_score(importance_type='weight')

# 全ての特徴量を取得してデータフレームにまとめる
all_features_xgb = X_train_split_xgb.columns
importance_df_xgb = pd.DataFrame({
    'Feature': all_features_xgb,
    'Weight': [importance_weight_xgb.get(feature, 0) for feature in all_features_xgb],  # Weightがない場合は0
})


# データフレームにまとめて表示
importance_df_xgb = pd.DataFrame({
    'Feature': list(importance_weight_xgb.keys()),
    'Weight': list(importance_weight_xgb.values()),
   
})

# Weightで上位13項目をソート
top_13_weight_df_xgb = importance_df_xgb.sort_values(by='Weight', ascending=False).head(13)


# 結果の表示
print("Top 13 Feature Importances by Weight:\n", top_13_weight_df_xgb)


# グラフの作成
plt.figure(figsize=(10,8))

# Weightのプロット
plt.subplot(1, 2, 2)
plt.barh(top_13_weight_df_xgb['Feature'], top_13_weight_df_xgb['Weight'], color='lightgreen')
plt.xlabel('Weight')
plt.title('Top 13 Feature Importances by Weight')
plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc,confusion_matrix

# テストデータをDMatrix形式に変換
dtest = xgb.DMatrix(X_test_xgb)
# テストデータでの予測確率を取得
y_scores_xgb = model_xgb.predict(dtest_xgb)

# ROC計算用のしきい値を決定（スコアの一意な値を使用）
thresholds_xgb = np.sort(np.unique(y_scores_xgb))

# 感度（TPR）と特異度（1 - FPR）を計算
tpr_list_xgb = []
specificity_list_xgb = []

for threshold in thresholds_xgb:
    # スコアをしきい値で2値化
    y_pred_xgb = (y_scores_xgb >= threshold).astype(int)
    
    # 混同行列を取得
    tn_xgb, fp_xgb, fn_xgb, tp_xgb = confusion_matrix(y_test_xgb, y_pred_xgb).ravel()
    
    # 感度と特異度を計算
    tpr_xgb = tp_xgb / (tp_xgb + fn_xgb) if (tp_xgb + fn_xgb) > 0 else 0  # 感度
    specificity_xgb = tn_xgb / (tn_xgb + fp_xgb) if (tn_xgb + fp_xgb) > 0 else 0  # 特異度
    
    # リストに保存
    tpr_list_xgb.append(tpr_xgb)
    specificity_list_xgb.append(specificity_xgb)

# AUCの計算
roc_auc_xgb = auc(1 - np.array(specificity_list_xgb), tpr_list_xgb)

# ROC曲線を描画
plt.figure(figsize=(8, 6))
plt.plot(np.array(specificity_list_xgb), tpr_list_xgb, label=f"ROC Curve (AUC = {roc_auc_xgb:.2f})", color="blue")
plt.plot([1, 0], [0, 1], color="gray", linestyle="--")  # 45度線
plt.xlim([1.0, 0.0])  # X軸を1から0に設定
plt.ylim([0.0, 1.0])  # Y軸を0から1に設定
plt.xlabel("Specificity ")
plt.ylabel("Sensitivity")
plt.title("ROC Curve (Specificity vs Sensitivity)")
plt.legend(loc="lower left")
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# テストデータでの予測
y_pred_xgb = model_xgb.predict(dtest_xgb)
# 二値分類のため、0.5を閾値として0または1に変換
y_pred_xgb = [1 if pred > 0.5 else 0 for pred in y_pred_xgb]

# 混同行列の計算
cm = confusion_matrix(y_test_xgb, y_pred_xgb)

# 混同行列の表示
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Positive"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# 混同行列の要素を使って感度と特異度を計算
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)  # 感度
specificity = tn / (tn + fp)  # 特異度

print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt

# 最初の決定木を可視化する
xgb.plot_tree(model_xgb, num_trees=0)
plt.show()
# 決定木の可視化設定
fig, ax = plt.subplots(figsize=(20, 10))  # 図のサイズを大きく設定

# Graphvizの属性を辞書形式で設定
graph_attr = {'rankdir': 'LR', 'size': '20,10'}  # 'size'は図のサイズを指定（単位はインチ）


xgb.plot_tree(model_xgb, num_trees=0, ax=ax, **{'graph_attr': graph_attr})  # フォントサイズを指定
plt.show()

# 図をファイルとして保存
fig.savefig('new_tree_visualization.png')
